In [1]:

from appgeopy import *  # Another custom package, likely for geospatial processing
from my_packages import *  # Custom package containing specific functions
import time

# ------------------------------------------------------------------------------
# Script Information
# ------------------------------------------------------------------------------
print("\n\n" + "-"*50)
print("Script : Compare Line-Of-Sight (LOS) displacement between Persistent Scatterers (PS) and GPS stations over time")
print("-"*50)

# ------------------------------------------------------------------------------
# Utility Functions
# ------------------------------------------------------------------------------

def log_time(start_time):
    """Log the elapsed time since start."""
    end = time.time()
    elapsed_time_seconds = round((end - start_time), 2)
    elapsed_time_minutes = round((elapsed_time_seconds / 60), 2)
    elapsed_time_hours = round((elapsed_time_seconds / 3600), 2)
    
    print(f"Processing time (seconds): {elapsed_time_seconds}")
    print(f"Processing time (minutes): {elapsed_time_minutes}")
    print(f"Processing time (hours): {elapsed_time_hours}")




--------------------------------------------------
Script : Compare Line-Of-Sight (LOS) displacement between Persistent Scatterers (PS) and GPS stations over time
--------------------------------------------------


In [ ]:
# Record the start time of the process for performance tracking
start = time.time()

# ------------------------------------------------------------------------------
# Argument Parser Configuration
# ------------------------------------------------------------------------------
# parser = argparse.ArgumentParser(description="INSSTACKPSI post-processing scripts")
# parser.add_argument("configfile", help="Path to the configuration file containing parameters")
# args = parser.parse_args()
# print(f"Configuration file is: {args.configfile}")

# ------------------------------------------------------------------------------
# Configuration File Reading and Setup
# ------------------------------------------------------------------------------
config_fpath = r"D:\VENV_PYTHON\insar_scriptflow\B__post_insstackpsi_config.ini"
config = configparser.ConfigParser()
config.read(config_fpath)

# ------------------------------------------------------------------------------
# Extract General Parameters from the Configuration File
# ------------------------------------------------------------------------------
workflow_step = config["GENERAL"]["workflow_step"]
mainfolder = config["GENERAL"]["insstackpsi_mainfolder"]

insstackpsi_fld = os.path.join(mainfolder, "13_INSSTACKPSI_inspsn")
post_insstackpsi_fld = os.path.join(mainfolder, "14_POST-PROCESSING")
config_fld = os.path.join(post_insstackpsi_fld, "configs")

# Ensure the config folder exists and save the used configuration file
os.makedirs(config_fld, exist_ok=True)
# data_io.save_configs(src_file=args.configfile, dest_folder=config_fld)

los_difference_savename = config["GPS_DIFFERENCE"]["los_difference_output"]
los_merge_savename = config["GPS_DIFFERENCE"]["los_merge_output"]

# ------------------------------------------------------------------------------
# Determine Paths and Filenames Based on Workflow Step
# ------------------------------------------------------------------------------
if workflow_step == 'calibration':
    o_displacement_fpath = os.path.join(post_insstackpsi_fld, "ras2pnt_MODIFIED_oChoushui_DISPLACEMENT_LOS_mm.pkl")
    los_difference_savepath = os.path.join(post_insstackpsi_fld, los_difference_savename)
    los_merge_savepath = os.path.join(post_insstackpsi_fld, los_merge_savename)
else:
    validation_fld = os.path.join(mainfolder, "15_VALIDATION")
    os.makedirs(validation_fld, exist_ok=True)
    input_disp_filename = config["VALIDATION"]["input_disp_filename"]
    o_displacement_fpath = os.path.join(validation_fld, input_disp_filename)
    los_difference_savename = "VALIDATE_" + los_difference_savename
    los_merge_savename = "VALIDATE_" + los_merge_savename
    los_difference_savepath = os.path.join(validation_fld, los_difference_savename)
    los_merge_savepath = os.path.join(validation_fld, los_merge_savename)

# ------------------------------------------------------------------------------
# Clean Up Old Output Files
# ------------------------------------------------------------------------------
for fpath in [los_difference_savepath, los_merge_savepath]:
    if os.path.exists(fpath):
        os.remove(fpath)
        print(f"Removed old file: {fpath}")

# ------------------------------------------------------------------------------
# Load and Configure Data
# ------------------------------------------------------------------------------
gps_los_difference_fpath = config["GPS_DIFFERENCE"]["gps_los_difference_fpath"]
radius_of_buffer = int(config["PCS_SEARCH"]["buffer_radius"])
input_mfile = config["GENERAL"]["input_mfile"]

try:
    o_displacement_geodf = pd.read_pickle(o_displacement_fpath, compression="zip").set_index("PointKey")
    print(f"Loaded displacement data from {o_displacement_fpath}")
except Exception as e:
    print(f"Error loading displacement data: {e}")
    raise

mfile_datetimes = datetime_handle.extract_datetime_from_mfile(input_mfile)[1:]

try:
    select_pointkey_fpath = glob(os.path.join(post_insstackpsi_fld, f"*{radius_of_buffer}*.pkl"))[0]
    select_pointkey = pd.read_pickle(select_pointkey_fpath, compression="zip")
    print(f"Loaded selected point keys from {select_pointkey_fpath}")
except IndexError:
    print("No matching file found for the given buffer radius.")
    raise
except Exception as e:
    print(f"Error loading selected point keys: {e}")
    raise

available_stations = select_pointkey["STATION"].unique()


In [ ]:

# ------------------------------------------------------------------------------
# Process Each GPS Station and Compare LOS Displacements
# ------------------------------------------------------------------------------
select_station = available_stations[0]
# for select_station in tqdm(available_stations, desc="Processing GPS stations"):
pointkey_by_station = select_pointkey.loc[select_station]["PointKey"]
psc_by_station = o_displacement_geodf.loc[pointkey_by_station, mfile_datetimes]

avg_deform_psc_by_station = pd.DataFrame(psc_by_station.mean(axis=0), columns=["LOS(mm)"])
avg_deform_psc_by_station.index = avg_deform_psc_by_station.index.map(datetime_handle.convert_to_datetime)

gps_los_difference_by_station = pd.read_excel(
    gps_los_difference_fpath,
    sheet_name=select_station,
    usecols=[0, 4],  # Use the first column as date and the fifth column as LOS difference
    parse_dates=[0],  # Parse the first column as dates
    index_col=[0]  # Set the first column as the index
)

mutual_index = datetime_handle.intersect_time_index(
    df1_index=avg_deform_psc_by_station.index,
    df2_index=gps_los_difference_by_station.index
)

los_difference, los_merge = ts_disp.compare_LOS_disp(
    psc_df=avg_deform_psc_by_station,
    gps_df=gps_los_difference_by_station,
    mutual_index=mutual_index
)

for dataframe in [los_merge, los_difference]:
    dataframe.reset_index(drop=False, inplace=True)
    dataframe.rename(columns={"index": "Datetime"}, inplace=True)

# data_io.save_df_to_excel(
#     df_to_save=los_difference,
#     filepath=los_difference_savepath,
#     sheet_name=select_station,
#     verbose=False
# )

# data_io.save_df_to_excel(
#     df_to_save=los_merge,
#     filepath=los_merge_savepath,
#     sheet_name=select_station,
#     verbose=False
# )